This file contains R functions that reproduce the analysis permormed in the continuous network from **Herrera et al. 2024. A system-Level Model Reveals that Transcriptional Stochasticity is Required for Hematopoietic Stem Cell Differentiation**

In [32]:
if (!requireNamespace("deSolve", quietly = TRUE)) {install.packages("deSolve")}
if (!requireNamespace("ggplot2", quietly = TRUE)) {install.packages("ggplot2")}
if (!requireNamespace("reshape2", quietly = TRUE)) {install.packages("reshape2")}
if (!requireNamespace("dplyr", quietly = TRUE)) {install.packages("dplyr")}
if (!requireNamespace("BoolNet", quietly = TRUE)) {install.packages("BoolNet")}
if (!requireNamespace("gridExtra", quietly = TRUE)) {install.packages("gridExtra")}
if (!requireNamespace("tidyr", quietly = TRUE)) {install.packages("tidyr")}
if (!requireNamespace("grid", quietly = TRUE)) {install.packages("grid")}

library(deSolve)
library(ggplot2)
library(reshape2)
library(dplyr)
library(BoolNet)
library(gridExtra)
library(tidyr)
library(grid)


In [33]:
# define network parameters
parameters <- c(h = 50,gamma = 1)

# define SQUAD generic function
SQUAD<-function(x,w,gamma,h){
  val<- ((-exp(0.5*h) + exp(-h*(w-0.5))) / ((1-exp(0.5*h)) * (1+exp(-h*(w-0.5))))) - (gamma*x)
  return(val)
}

# ODE system
squadInteractions= function (t, state, parameters) {
    with(as.list(c(state, parameters)), {
               w_Oxygen = Oxygen
        w_Runx1 = min(max(max(PU1, GATA2), Runx1), 1 - Ikzf1)
        w_Meis1 = min(max(max(Meis1, Runx1), PU1), 1 - Gfi1)
        w_HIF1 = max(min(1 - Oxygen, Meis1), min(min(1 - Oxygen, 
            O2), 1 - max(max(p53, FOXO3), AMPK)))
        w_FOXO3 = min(min(AMPK, max(HIF1, p53)), 1 - max(AKT, 
            Cebpa))
        w_p53 = min(HIF1, 1 - max(max(AKT, GATA1), PU1))
        w_GATA2 = min(max(GATA2, p53), 1 - max(max(GATA1, PU1), 
            Gfi1))
        w_GATA1 = min(max(max(max(GATA1, GATA2), AKT), Runx1), 
            1 - max(max(PU1, p53), Ikzf1))
        w_PU1 = min(max(max(PU1, Runx1), min(Cebpa, Ikzf1)), 
            1 - max(max(GATA1, GATA2), Gfi1))
        w_Cebpa = min(min(PU1, Runx1), 1 - Mef2c)
        w_Ikzf1 = min(max(max(Mef2c, Ikzf1), Runx1), 1 - max(max(Cebpa, 
            PU1), GATA1))
        w_Gfi1 = min(max(Ikzf1, Cebpa), 1 - max(PU1, p53))
        w_Mef2c = min(PU1, 1 - Cebpa)
        w_mTOR = min(AKT, min(1 - AMPK, 1 - p53))
        w_AMPK = min(max(max(1 - AKT, 1 - OXPHOS), 1 - HIF1), 
            p53)
        w_AKT = max(H2O2, 1 - max(p53, FOXO3))
        w_H2O2 = min(max(max(max(max(GATA1, HIF1), PU1), OXPHOS), 
            min(SOD, O2)), 1 - Antiox)
        w_O2 = max(1 - SOD, OXPHOS)
        w_SOD = max(p53, FOXO3)
        w_Antiox = max(p53, FOXO3)
        w_OXPHOS = min(max(mTOR, AKT), 1 - max(FOXO3, HIF1))
		
       # Rates of Change
		dOxygen =SQUAD(Oxygen,w_Oxygen,gamma,h)
    	dRunx1 = SQUAD(Runx1,w_Runx1,gamma,h)
    	dMeis1 = SQUAD(Meis1,w_Meis1,gamma,h)
    	dHIF1 = SQUAD(HIF1,w_HIF1,gamma,h)
    	dFOXO3 = SQUAD(FOXO3,w_FOXO3,gamma,h)
    	dp53 = SQUAD(p53,w_p53,gamma,h)
    	dGATA2 = SQUAD(GATA2,w_GATA2,gamma,h)
    	dGATA1 = SQUAD(GATA1,w_GATA1,gamma,h)
    	dPU1 = SQUAD(PU1,w_PU1,gamma,h)
    	dCebpa = SQUAD(Cebpa,w_Cebpa,gamma,h)
    	dIkzf1 = SQUAD(Ikzf1,w_Ikzf1,gamma,h)
    	dGfi1 = SQUAD(Gfi1,w_Gfi1,gamma,h)
    	dMef2c = SQUAD(Mef2c,w_Mef2c,gamma,h)
    	dmTOR = SQUAD(mTOR,w_mTOR,gamma,h)
    	dAMPK = SQUAD(AMPK,w_AMPK,gamma,h)
    	dAKT = SQUAD(AKT,w_AKT,gamma,h)
    	dH2O2 = SQUAD(H2O2,w_H2O2,gamma,h)
    	dO2 = SQUAD(O2,w_O2,gamma,h)
    	dSOD = SQUAD(SOD,w_SOD,gamma,h)
    	dAntiox = SQUAD(Antiox,w_Antiox,gamma,h)
    	dOXPHOS = SQUAD(OXPHOS,w_OXPHOS,gamma,h)
		
       return(list(c(dOxygen, dRunx1, dMeis1, dHIF1, dFOXO3, dp53, 
            dGATA2, dGATA1, dPU1, dCebpa, dIkzf1, dGfi1, dMef2c, 
            dmTOR, dAMPK, dAKT, dH2O2, dO2, dSOD, dAntiox, 
            dOXPHOS)))
    })
}

In [34]:

if (!requireNamespace("doParallel", quietly = TRUE)) {install.packages("doParallel")}

library(doParallel)
library(foreach)
library(compiler)


samples <- 1000

# Function to assign random boolean values to the init list
randomize_init <- cmpfun(function() {
  genes <- c(
    "Oxygen", "Runx1", "Meis1", "HIF1", "FOXO3", "p53", "GATA2", "GATA1", 
    "PU1", "Cebpa", "Ikzf1", "Gfi1", "Mef2c", "mTOR", "AMPK", "AKT", 
    "H2O2", "O2", "SOD", "Antiox", "OXPHOS"
  )
  
  init <- setNames(sample(c(0, 1), length(genes), replace = TRUE), genes)
  return(init)
})

# Create a matrix to store initial states
initial_states <- matrix(nrow = samples, ncol = 21)
colnames(initial_states) <- c(
  "Oxygen", "Runx1", "Meis1", "HIF1", "FOXO3", "p53", "GATA2", "GATA1", 
  "PU1", "Cebpa", "Ikzf1", "Gfi1", "Mef2c", "mTOR", "AMPK", "AKT", 
  "H2O2", "O2", "SOD", "Antiox", "OXPHOS"
)

# Load the boolean network
boolean_network <- loadNetwork("BoolNet_HSC_network.txt")

# Populate the matrix with random initial states
for (i in 1:samples) {
  initial_value <- randomize_init()
  initial_states[i, ] <- generateState(boolean_network, specs = initial_value)
}

# Define interval for numeric integration
times <- seq(0, 10, 0.005)

# Initialize matrix to store final states
final_states <- matrix(NA, nrow = samples, ncol = 21)

# Get the start time
start_time <- Sys.time()
cat("Process started at:", format(start_time, "%Y-%m-%d %H:%M:%S"), "\n")

# Set up parallel backend
num_cores <- detectCores() - 1
cl <- makeCluster(num_cores)
registerDoParallel(cl)

# Parallel simulation
final_states <- foreach(i = 1:samples, .combine = rbind, .packages = c("deSolve")) %dopar% {
  result <- ode(y = initial_states[i, ], times = times, func = squadInteractions, parms = parameters, atol = 10e-6, rtol = 10e-6)
  result[nrow(result), -1]  # store the final state at t=10
}

# Stop the cluster
stopCluster(cl)

# Step 4: Define the 11 pre-identified stable states
stable_states <- matrix(c(
  0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,1,0,0,0, # 1 MEP
  1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,1,0,0,1, # 2 MEP
  0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,1,1,1,0,0,0, # 3 MEP
  1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,1,1,0,0,1, # 4 MEP
  0,1,1,1,0,0,0,1,0,0,0,0,0,1,0,1,1,1,0,0,0, # 5 MEP
  1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1,1,1,0,0,1, # 6 MEP
  0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,1,1,1,0,0,0, # 7 CLP
  1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,1,1,0,0,1, # 8 CLP
  0,1,1,1,0,0,0,0,1,0,0,0,1,1,0,1,1,1,0,0,0, # 9 GMP
  1,1,1,0,0,0,0,0,1,0,0,0,1,1,0,1,1,1,0,0,1, # 10 GMP
  0,1,1,1,0,0,0,0,1,1,0,0,0,1,0,1,1,1,0,0,0, # 11 GMP
  1,1,1,0,0,0,0,0,1,1,0,0,0,1,0,1,1,1,0,0,1, # 12 GMP
  0,0,1,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0  # 13 HSC
), ncol = 21, byrow = TRUE)

# Function to calculate Euclidean distance
euclidean_distance <- cmpfun(function(a, b) {
  sqrt(sum((a - b)^2))
})

# Compare a final state with the stable states
compare_with_stable_states <- function(final_state, stable_states) {
  distances <- apply(stable_states, 1, function(stable_state) euclidean_distance(final_state, stable_state))
  closest_state_index <- which.min(distances)
  closest_state <- stable_states[closest_state_index, ]
  list(closest_state = closest_state, distance = min(distances), closest_state_index = closest_state_index)
}

# Step 5: Evaluate All Final States
threshold <- 0.1  # Define the proximity threshold
state_counts <- integer(nrow(stable_states))  # Counter for stable states
other_state_count <- 0  # Counter for other unidentified stable states

for (i in 1:nrow(final_states)) {
  comparison <- compare_with_stable_states(final_states[i, ], stable_states)
  if (comparison$distance <= threshold) {
    state_counts[comparison$closest_state_index] <- state_counts[comparison$closest_state_index] + 1
  } else {
    other_state_count <- other_state_count + 1
  }
}

# Define labels and colors for the barplot
labels <- c("1 MEP","2 MEP","3 MEP","4 MEP","5 MEP","6 MEP", "7 CLP","8 CLP",  "9 GMP", "10 GMP","11 GMP", "12 GMP","13 HSC", "Others")
colors <- c(
  rep("#E5007E", 6),  # MEP
  rep("#4F5AA4", 2),  # CLP
  rep("#B185BA", 4),  # GMP
  "#800080",          # HSC
  "#D9D9D9"           # Others
)

# Create a histogram of the results
pdf("Continuos_Attractors_frequency.pdf", width = 11, height = 8.5)
barplot(c(state_counts, other_state_count), 
        names.arg = labels, 
        main = paste("Attractors frequency from", samples, "random initial states"), 
        xlab = "Attractor", 
        ylab = "Frequency", 
        col = colors, 
        las = 2)  # Rotate x-axis labels for better visibility

# Create and display a dataframe called frequency_df combining state_counts and other_state_count
frequency_df <- data.frame(
  State = c(paste("Atractor", 1:nrow(stable_states)), "Others"),
  Frequency = c(state_counts, other_state_count)
) %>%
  mutate(
    Relative_Frequency = (Frequency / samples) * 100,
    Norm_Relative_Frequency = c((Frequency[1:13] / sum(Frequency[1:13])) * 100, rep(NA, nrow(.) - 13))
)

# Display the table
grid.newpage()
grid.text(paste("Attractors frequency from ", samples, " random initial states "), 
          gp = gpar(fontsize = 20, fontface = "bold"), 
          y = unit(0.9, "npc")
)
grid.table(frequency_df)

# Add a new page for the table
grid.newpage()
grid.text(paste("Cumulative frequency from ", samples, " random initial states "), 
          gp = gpar(fontsize = 20, fontface = "bold"), 
          y = unit(0.9, "npc")
)
grid.table(data.frame(
  Identity = c("MEP", "CLP", "GMP", "HSC", "Total"),
  cumulative_frequency = c(
    sum(frequency_df$Norm_Relative_Frequency[1:6]), 
    sum(frequency_df$Norm_Relative_Frequency[7:8]), 
    sum(frequency_df$Norm_Relative_Frequency[9:12]), 
    sum(frequency_df$Norm_Relative_Frequency[13]), 
    sum(frequency_df$Norm_Relative_Frequency[1:13])
  )
))
dev.off()
